In [1]:
# 开始数据迁移工作了
import os
import django
os.environ.setdefault('IS_DEBUG', 'TRUE')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'main.settings')
django.setup()

我现在是一名AI开发工程师，现在实现图搜图，文搜图的功能，请帮我部署Qwen3-VL-4B/8B，并生成对应的对指定图片进行调用生成图片特征向量的逻辑代码。
1. 安装依赖库
首先安装所需的 Python 库：

In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("image-text-to-text", model="Qwen/Qwen3-VL-4B-Instruct")
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "url": "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG"},
            {"type": "text", "text": "What animal is on the candy?"}
        ]
    },
]
pipe(text=messages)

/Users/calvin/projs/ModePlanAI/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /Qwen/Qwen3-VL-4B-Instruct/resolve/main/config.json (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))"), '(Request ID: beff51a6-7d10-4378-9468-c00568950868)')' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen3-VL-4B-Instruct/resolve/main/config.json
Retrying in 1s [Retry 1/5].
Fetching 2 files:   0%|          | 0/2 [3:48:41<?, ?it/s]
Cancellation requested; stopping current tasks.


KeyboardInterrupt: 

In [ ]:
from transformers import Qwen3VLForConditionalGeneration

In [1]:
# 2. 加载模型和处理器
# 使用 Hugging Face Transformers 库加载 Qwen3-VL 模型：
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor
import torch

# 加载模型和处理器
model_path = "Qwen/Qwen3-VL-4B-Instruct"  # 或 "Qwen/Qwen2-VL-8B"
model = Qwen3VLForConditionalGeneration.from_pretrained(
    model_path,
    dtype=torch.float16,
    device_map="auto"
)
processor = AutoProcessor.from_pretrained(model_path)


/Users/calvin/projs/ModePlanAI/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /Qwen/Qwen3-VL-4B-Instruct/resolve/main/config.json (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))"), '(Request ID: 8831a170-0a19-465a-98e4-169ebd1a294b)')' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen3-VL-4B-Instruct/resolve/main/config.json
Retrying in 1s [Retry 1/5].
Fetching 2 files:   0%|          | 0/2 [00:12<?, ?it/s]
Cancellation requested; stopping current tasks.


KeyboardInterrupt: 

In [ ]:
# 3. 图像预处理和特征提取
# 以下代码展示如何对指定图像生成特征向量：
from PIL import Image
import requests
import torch

def generate_image_features(image_path_or_url):
    """
    对指定图像生成特征向量

    Args:
        image_path_or_url (str): 图像路径或URL

    Returns:
        torch.Tensor: 图像特征向量
    """
    # 加载图像
    if image_path_or_url.startswith("http"):
        image = Image.open(requests.get(image_path_or_url, stream=True).raw)
    else:
        image = Image.open(image_path_or_url)

    # 处理图像
    inputs = processor(
        images=image,
        return_tensors="pt"
    ).to(model.device)

    # 生成特征向量
    with torch.no_grad():
        # 获取视觉编码器的输出作为特征
        vision_outputs = model.vision_tower(inputs.pixel_values)
        # 可以使用最后一个隐藏状态的平均池化作为特征向量
        features = vision_outputs.last_hidden_state.mean(dim=1)

    return features

# 使用示例
image_path = "path/to/your/image.jpg"
features = generate_image_features(image_path)
print(f"Feature vector shape: {features.shape}")


In [ ]:
# 4. 批量处理图像
# 如果您需要批量处理多张图像：
def batch_generate_features(image_paths):
    """
    批量生成图像特征向量

    Args:
        image_paths (list): 图像路径列表

    Returns:
        torch.Tensor: 批量图像特征向量
    """
    images = []
    for path in image_paths:
        if path.startswith("http"):
            img = Image.open(requests.get(path, stream=True).raw)
        else:
            img = Image.open(path)
        images.append(img)

    # 批量处理
    inputs = processor(
        images=images,
        return_tensors="pt"
    ).to(model.device)

    with torch.no_grad():
        vision_outputs = model.vision_tower(inputs.pixel_values)
        features = vision_outputs.last_hidden_state.mean(dim=1)

    return features

# 使用示例
image_list = ["image1.jpg", "image2.jpg", "image3.jpg"]
batch_features = batch_generate_features(image_list)
print(f"Batch features shape: {batch_features.shape}")


In [ ]:
# 5. 特征存储和检索
# 为实现图搜图功能，您可以将特征向量存储在向量数据库中：
import numpy as np
import faiss

# 存储特征向量到FAISS索引
def create_vector_index(features_list):
    """
    创建向量索引用于相似度搜索

    Args:
        features_list (list): 特征向量列表

    Returns:
        faiss.Index: FAISS索引对象
    """
    # 转换为numpy数组
    features_array = torch.stack(features_list).cpu().numpy()

    # 创建FAISS索引
    dimension = features_array.shape[1]
    index = faiss.IndexFlatIP(dimension)  # 内积相似度

    # 归一化向量
    faiss.normalize_L2(features_array)

    # 添加到索引
    index.add(features_array.astype(np.float32))

    return index

# 相似图像搜索
def search_similar_images(query_feature, index, k=5):
    """
    搜索相似图像

    Args:
        query_feature (torch.Tensor): 查询图像特征
        index (faiss.Index): FAISS索引
        k (int): 返回最相似的k个结果

    Returns:
        tuple: (距离, 索引)
    """
    # 转换并归一化查询特征
    query_vector = query_feature.cpu().numpy().reshape(1, -1)
    faiss.normalize_L2(query_vector)

    # 搜索相似向量
    distances, indices = index.search(query_vector.astype(np.float32), k)

    return distances[0], indices[0]


In [ ]:
# 6. 文本到图像搜索
# 对于文本搜索图像功能，您可以使用模型的文本编码器：
def generate_text_features(text_prompt):
    """
    生成文本特征向量

    Args:
        text_prompt (str): 文本提示

    Returns:
        torch.Tensor: 文本特征向量
    """
    # 编码文本
    inputs = processor(
        text=text_prompt,
        return_tensors="pt"
    ).to(model.device)

    # 获取文本特征
    with torch.no_grad():
        # 使用文本编码器获取特征
        text_outputs = model.language_model.model.embed_tokens(inputs.input_ids)
        features = text_outputs.mean(dim=1)

    return features

# 文本搜索图像示例
query_text = "一只可爱的猫咪"
text_features = generate_text_features(query_text)
distances, indices = search_similar_images(text_features, image_index, k=5)


In [ ]:
# 7. Django集成
# 根据您的Django环境设置，可以创建一个视图来处理图像特征生成：
# views.py
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
import json

@csrf_exempt
def generate_features_view(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        image_path = data.get('image_path')

        try:
            features = generate_image_features(image_path)
            features_list = features.cpu().numpy().tolist()

            return JsonResponse({
                'status': 'success',
                'features': features_list
            })
        except Exception as e:
            return JsonResponse({
                'status': 'error',
                'message': str(e)
            })
